In [1]:
import torch

torch.__version__

'2.0.0+cu118'

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device=', device)

device= cuda


In [19]:
from transformers import BertModel

#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-uncased')
#需要移动到cuda上
pretrained.to(device)

#不训练,不需要计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)


#定义下游任务模型
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.linear1 = torch.nn.Linear(1,16)
        self.linear2 = torch.nn.Linear(1,16)
        self.linear3 = torch.nn.Linear(1,16)
        self.linear4 = torch.nn.Linear(1,16)

        self.fc = torch.nn.Linear(768+3, 2)

    def forward(self, input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission):
        with torch.no_grad():
            out = pretrained(input_ids=input_ids,
                       attention_mask=attention_mask,
                       token_type_ids=token_type_ids)
        # print(code_len)
        # code_len = self.linear1(code_len)
        # keyword_num = self.linear2(keyword_num)
        # keyword_fix = self.linear3(keyword_fix)
        # permission = self.linear4(permission)

        # code_len = code_len
        # keyword_num = keyword_num.unsqueeze(1)
        # keyword_fix = keyword_fix.unsqueeze(1)
        # permission = permission.unsqueeze(1)
        # code_len = torch.transpose(code_len, 0, 1)
        # keyword_num = torch.transpose(keyword_num, 0, 1)
        # keyword_fix = torch.transpose(keyword_fix, 0, 1)
        # permission = torch.transpose(permission, 0, 1)


        concat = torch.cat((out.last_hidden_state[:, 0], code_len, keyword_num, permission), dim=1)
        out = self.fc(concat)

        out = out.softmax(dim=1)

        return out


model = Model()
#同样要移动到cuda
model.to(device)

#虚拟一批数据,需要把所有的数据都移动到cuda上
input_ids = torch.ones(16, 100).int().to(device)
attention_mask = torch.ones(16, 100).int().to(device)
token_type_ids = torch.ones(16, 100).int().to(device)
code_len = torch.ones(16,1).float().to(device)
keyword_num = torch.ones(16,1).float().to(device)
permission = torch.ones(16,1).float().to(device)
labels = torch.ones(16,1).float().to(device)

#试算
model(input_ids=input_ids,
      attention_mask=attention_mask,
      token_type_ids=token_type_ids,
      code_len=code_len,
      keyword_num=keyword_num,
      permission=permission).shape

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


torch.Size([16, 2])

In [20]:
from datasets import load_dataset
from datasets import load_from_disk

#定义数据集
class Dataset:
    def __init__(self):
        self.dataset = load_dataset(path='csv', data_dir='D:\\hzrproject\\study\\non-cross study\\2',data_files='modified_issueDataNoTensorflow.csv')['train']


    def __len__(self):

        return len(self.dataset)

    def __getitem__(self, i):
        ds = self.dataset[i]['ds']
        code_len = self.dataset[i]['code_len']
        keyword_num = self.dataset[i]['keyword_num']
        keyword_fix = self.dataset[i]['keyword_fix']
        permission = self.dataset[i]['permission']
        label = self.dataset[i]['label']

        return  ds, code_len, keyword_num, keyword_fix, permission, label

class Dataset_validation:
    def __init__(self):
        self.dataset_validation = load_dataset(path='csv',  data_dir='D:\\hzrproject\\study\\non-cross study\\2',data_files='modified_issueDataTensorflow_feature3.csv')['train']

    def __len__(self):
        return len(self.dataset_validation)

    def __getitem__(self, i):
        ds = self.dataset_validation[i]['ds']
        code_len = self.dataset_validation[i]['code_len']
        keyword_num = self.dataset_validation[i]['keyword_num']
        keyword_fix = self.dataset_validation[i]['keyword_fix']
        permission = self.dataset_validation[i]['permission']
        label = self.dataset_validation[i]['label']

        return  ds, code_len, keyword_num, keyword_fix, permission, label


dataset = Dataset()
dataset_validation = Dataset_validation()

len(dataset), dataset[0], len(dataset_validation), dataset_validation[0]

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/hananan/.cache/huggingface/datasets/csv/default-08fbfe6a6745fbd4/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


D:\python\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/hananan/.cache/huggingface/datasets/csv/default-04f380cb9cfc9313/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


D:\python\lib\site-packages\datasets\download\streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


  0%|          | 0/1 [00:00<?, ?it/s]

(2396,
 ('title:Jacoco plugin causes implicit dependencies between unrelated tasks description:<!--- Please follow the instructions below. We receive dozens of issues every week, so to stay productive, we will close issues that don\'t provide enough information. Please open Android-related issues on the Android Issue Tracker at https://source.android.com/source/report-bugsPlease open Gradle Native-related issues at https://github.com/gradle/gradle-native/issues--><!--- Provide a brief summary of the issue in the title above -->### Expected BehaviorWe\'re applying the jacoco plugin to multiple tasks within a single project, all of type JavaExec or JavaExecFork (psxpaul external plugin) like so:```jacoco {    applyTo initTestBDeploy    applyTo checkTestBDeploy    applyTo startTestBDeploy    if(!project.hasProperty(\'cypressNoCentral\')) {        applyTo startTestBDeployCentral        applyTo startTestBDeployManaged    }}```Expectation is that these tasks maintain their "current" relation

In [21]:
from transformers import BertTokenizer

#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-uncased')


def collate_fn(data):
    sents = [i[0] for i in data]
    code_len = [i[1] for i in data]
    keyword_num = [i[2] for i in data]
    permission = [i[4] for i in data]
    labels = [i[5] for i in data]

    #编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=sents,
                                   truncation=True,
                                   padding='max_length',
                                   max_length=500,
                                   return_tensors='pt',
                                   return_length=True)

    #input_ids:编码之后的数字
    #attention_mask:是补零的位置是0,其他位置是1
    input_ids = data['input_ids'].to(device)
    attention_mask = data['attention_mask'].to(device)
    token_type_ids = data['token_type_ids'].to(device)

    code_len = torch.FloatTensor(code_len).to(device)
    code_len = code_len.unsqueeze(0).t()

    keyword_num = torch.FloatTensor(keyword_num).to(device)
    keyword_num = keyword_num.unsqueeze(0).t()

    permission = torch.FloatTensor(permission).to(device)
    permission = permission.unsqueeze(0).t()

    labels = torch.LongTensor(labels).to(device).to(device)

    #print(data['length'], data['length'].max())

    return input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission, labels


#数据加载器
loader = torch.utils.data.DataLoader(dataset=dataset,
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission,
        labels) in enumerate(loader):
    break

print(len(loader))
input_ids.shape, attention_mask.shape, token_type_ids.shape, code_len.shape, keyword_num.shape, permission.shape, labels

149


(torch.Size([16, 500]),
 torch.Size([16, 500]),
 torch.Size([16, 500]),
 torch.Size([16, 1]),
 torch.Size([16, 1]),
 torch.Size([16, 1]),
 tensor([0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], device='cuda:0'))

In [22]:
from transformers import AdamW

#训练
optimizer = AdamW(model.parameters(), lr=2e-5)

criterion = torch.nn.CrossEntropyLoss()

model.train()
for epoch in range(40):
    all_right = 0
    all_label = 0
    print(f"epoch: {epoch+1}")
    for i, (input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission,
            labels) in enumerate(loader):
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    code_len=code_len,
                    keyword_num=keyword_num,
                    permission=permission)

        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if i % 10 == 0:

            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)

            all_right = all_right + (out == labels).sum().item()
            all_label = all_label + len(labels)
            print(i, loss.item(), accuracy)
        if i == 1000:
            break

    print(all_right/all_label)
    print("----------------------")



D:\python\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


epoch: 1
0 0.8045823574066162 0.5
10 0.7754069566726685 0.5
20 0.7055902481079102 0.5625
30 0.734526515007019 0.375
40 0.7538121342658997 0.5
50 0.7638300657272339 0.625
60 0.6858932375907898 0.625
70 0.5539385676383972 0.6875
80 0.7440857887268066 0.6875
90 0.7759900093078613 0.4375
100 0.7758898138999939 0.5
110 0.6311805844306946 0.625
120 0.5965038537979126 0.6875
130 0.7123362421989441 0.5
140 0.8692761659622192 0.4375
0.55
----------------------
epoch: 2
0 0.7798970937728882 0.5
10 0.8861294388771057 0.25
20 0.7917545437812805 0.4375
30 0.7390800714492798 0.5625
40 0.8200836181640625 0.375
50 0.6587197780609131 0.625
60 0.7104282379150391 0.5
70 0.7075632810592651 0.5
80 0.7320884466171265 0.4375
90 0.6530802845954895 0.6875
100 0.6600080132484436 0.6875
110 0.6904046535491943 0.5625
120 0.6965038180351257 0.4375
130 0.7739806175231934 0.375
140 0.6963897347450256 0.5625
0.5
----------------------
epoch: 3
0 0.6560837030410767 0.75
10 0.687950849533081 0.5625
20 0.631830930709838

In [23]:
#测试
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import precision_recall_fscore_support, roc_auc_score

def test():
    model.eval()
    correct = 0
    total = 0

    loader_test = torch.utils.data.DataLoader(dataset=dataset_validation,
                                              batch_size=10,
                                              collate_fn=collate_fn,
                                              shuffle=True,
                                              drop_last=True)
    y_true = []
    y_pred = []

    for i, (input_ids, attention_mask, token_type_ids, code_len, keyword_num, permission,
            labels) in enumerate(loader_test):

        if i == 20:
            break

        print(i)

        with torch.no_grad():
            out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    code_len=code_len,
                    keyword_num=keyword_num,
                    permission=permission)

        out = out.argmax(dim=1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(out.cpu().numpy())

        correct += (out == labels).sum().item()
        total += len(labels)

    # print(y_true)
    # print(y_pred)
    accuracy = correct / total
    precision, recall, f1_score, _ = precision_recall_fscore_support(y_true, y_pred, pos_label=1, average='binary')
    auc_score = roc_auc_score(y_true, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1_score}")
    print(f"AUC score: {auc_score}")


test()

0
1
2
3
4
5
6
7
8
9
Accuracy: 0.52
Precision: 0.5121951219512195
Recall: 0.84
F1-score: 0.6363636363636365
AUC score: 0.52
